In [1]:
import pandas as pd
from scripts.processing import process_datasets, load_datasets
from scripts.rf5_scorer import rf5_scorer, simulate_rf5_scorer

In [2]:
process_datasets(
    categories_path='data/categories.json',
    ballots_csv_path='data/_local/rf5_votes_final.csv',
    voters_csv_path='data/_local/rf5_voters.csv',
    voter_survey_csv_path='data/_local/rf5_survey.csv',
    projects_csv_path='data/_local/rf5_projects.csv',
    output_dir='data/_local/clean'
)
voting_data, df_all_votes, df_voters, df_projects = load_datasets('data/_local/clean')

Processing complete.


In [3]:
ballot_summary = []
for ballot in voting_data:
    ballot_dict = {
        'Voter Address': ballot['voter_address'],
        'Budget': ballot['budget']
    }
    for cat in ballot['category_allocations']:
        cat_key = list(cat.keys())[0]
        cat_val = list(cat.values())[0]
        ballot_dict.update({cat_key:cat_val})
    ballot_summary.append(ballot_dict)

df_summary = pd.DataFrame(ballot_summary)
df_summary.to_csv("data/_local/exports/rf5_results_voter_breakdown.csv")

In [4]:
distribution_sims = [
    {'data': rf5_scorer(voting_data),
     'title': 'Overall distribution'},
    {'data': simulate_rf5_scorer(voting_data, df_voters[df_voters['is_expert']].index),
     'title': "Experts' distribution"},
    {'data': simulate_rf5_scorer(voting_data, df_voters[~df_voters['is_expert']].index),
     'title': "Non-experts' distribution"},
]
dists = []
cols = []
for d in distribution_sims:
    dists.append(d['data'])
    cols.append(d['title'])
df_dists = pd.concat(dists, axis=1)    
df_dists.columns = cols
(
    df_projects[['project_display_name', 'project_category']]
).join(df_dists).to_csv("data/_local/exports/rf5_results_final.csv")